In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import json
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# -------------------- Step 1: 데이터 로드 및 전처리 --------------------
data = []
with open('/content/drive/MyDrive/review_business_5up_with_text.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame(data)
df = df[['user_id', 'business_id', 'stars', 'text']]

user2idx = {uid: i for i, uid in enumerate(df['user_id'].unique())}
item2idx = {iid: i for i, iid in enumerate(df['business_id'].unique())}
df['user'] = df['user_id'].map(user2idx)
df['item'] = df['business_id'].map(item2idx)

# -------------------- Step 2: SBERT 문맥 벡터 생성 --------------------
sbert = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
context_vectors = sbert.encode(df['text'].tolist(), show_progress_bar=True)
df['context_vector'] = list(context_vectors)

# -------------------- Step 3: Dataset 클래스 정의 --------------------
class UCAMDataset(Dataset):
    def __init__(self, users, items, ratings, contexts):
        self.users = users
        self.items = items
        self.ratings = ratings
        self.contexts = contexts

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.users[idx], dtype=torch.long),
            torch.tensor(self.items[idx], dtype=torch.long),
            torch.tensor(self.contexts[idx], dtype=torch.float32),
            torch.tensor(self.ratings[idx], dtype=torch.float32)
        )

# -------------------- Step 4: 데이터 분할 --------------------
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = UCAMDataset(train_df['user'].values, train_df['item'].values,
                            train_df['stars'].values, np.stack(train_df['context_vector']))
test_dataset = UCAMDataset(test_df['user'].values, test_df['item'].values,
                           test_df['stars'].values, np.stack(test_df['context_vector']))

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

# -------------------- Step 5: 모델 정의 --------------------
class UCAM(nn.Module):
    def __init__(self, num_users, num_items, context_dim=384, embed_dim=64):
        super().__init__()
        self.user_embed = nn.Embedding(num_users, embed_dim)
        self.item_embed = nn.Embedding(num_items, embed_dim)
        self.fc1 = nn.Linear(embed_dim * 2 + context_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, user_ids, item_ids, context_vecs):
        u = self.user_embed(user_ids)
        i = self.item_embed(item_ids)
        x = torch.cat([u, i, context_vecs], dim=-1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x).squeeze()

# -------------------- Step 6: 모델 학습 + 검증 + 조기 종료 --------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UCAM(num_users=len(user2idx), num_items=len(item2idx)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

best_val_rmse = float('inf')
epochs_no_improve = 0
patience = 5
min_delta = 0.001
epochs = 50
model_path = 'best_ucam_model.pt'

print("모델 학습 시작...")

for epoch in range(epochs):
    model.train()
    total_train_loss = 0

    for user_ids, business_ids, sentiment_vectors, stars in train_loader:
        user_ids = user_ids.to(device)
        business_ids = business_ids.to(device)
        sentiment_vectors = sentiment_vectors.to(device)
        stars = stars.to(device)

        optimizer.zero_grad()
        predictions = model(user_ids, business_ids, sentiment_vectors)
        loss = criterion(predictions, stars)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # --- 검증 ---
    model.eval()
    val_preds, val_true = [], []
    with torch.no_grad():
        for user_ids, business_ids, sentiment_vectors, stars in test_loader:
            user_ids = user_ids.to(device)
            business_ids = business_ids.to(device)
            sentiment_vectors = sentiment_vectors.to(device)
            stars = stars.to(device)

            preds = model(user_ids, business_ids, sentiment_vectors)
            val_preds.extend(preds.cpu().numpy())
            val_true.extend(stars.cpu().numpy())

    val_preds = np.array(val_preds)
    val_true = np.array(val_true)
    val_mse = mean_squared_error(val_true, val_preds)
    val_rmse = np.sqrt(val_mse)
    val_mae = mean_absolute_error(val_true, val_preds)
    val_mape = np.mean(np.abs((val_true - val_preds) / (val_true + 1e-10))) * 100

    print(f"\nEpoch {epoch+1} | Train Loss: {total_train_loss / len(train_loader):.4f} | "
          f"Val MSE: {val_mse:.4f}, RMSE: {val_rmse:.4f}, MAE: {val_mae:.4f}, MAPE: {val_mape:.2f}%")

    if val_rmse < best_val_rmse - min_delta:
        best_val_rmse = val_rmse
        epochs_no_improve = 0
        torch.save(model.state_dict(), model_path)
        print(f"  --> 개선됨. 모델 저장됨 (RMSE: {best_val_rmse:.4f})")
    else:
        epochs_no_improve += 1
        print(f"  --> 개선 없음. ({epochs_no_improve}/{patience})")
        if epochs_no_improve == patience:
            print("조기 종료 발생.")
            break

# -------------------- Step 7: 테스트셋 평가 --------------------
def evaluate_model(model, data_loader, device):
    model.eval()
    preds, targets = [], []

    with torch.no_grad():
        for users, items, contexts, ratings in data_loader:
            users = users.to(device)
            items = items.to(device)
            contexts = contexts.to(device)
            ratings = ratings.to(device)

            output = model(users, items, contexts)
            preds.extend(output.cpu().numpy())
            targets.extend(ratings.cpu().numpy())

    preds = np.array(preds)
    targets = np.array(targets)

    mae = mean_absolute_error(targets, preds)
    mse = mean_squared_error(targets, preds)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((targets - preds) / (targets + 1e-10))) * 100

    print(f"\n✅ [UCAM] 최종 테스트 평가 지표:")
    print(f"   - MSE  : {mse:.4f}")
    print(f"   - RMSE : {rmse:.4f}")
    print(f"   - MAE  : {mae:.4f}")
    print(f"   - MAPE : {mape:.2f}%")

if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))
    print(f"최적 모델 로드 완료: {model_path}")

evaluate_model(model, test_loader, device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/13994 [00:00<?, ?it/s]

모델 학습 시작...

Epoch 1 | Train Loss: 0.9496 | Val MSE: 0.6885, RMSE: 0.8298, MAE: 0.6568, MAPE: 24.46%
  --> 개선됨. 모델 저장됨 (RMSE: 0.8298)

Epoch 2 | Train Loss: 0.6529 | Val MSE: 0.6584, RMSE: 0.8114, MAE: 0.6423, MAPE: 23.44%
  --> 개선됨. 모델 저장됨 (RMSE: 0.8114)

Epoch 3 | Train Loss: 0.5948 | Val MSE: 0.6187, RMSE: 0.7866, MAE: 0.6150, MAPE: 22.16%
  --> 개선됨. 모델 저장됨 (RMSE: 0.7866)

Epoch 4 | Train Loss: 0.5306 | Val MSE: 0.5976, RMSE: 0.7731, MAE: 0.6007, MAPE: 21.42%
  --> 개선됨. 모델 저장됨 (RMSE: 0.7731)

Epoch 5 | Train Loss: 0.4867 | Val MSE: 0.5937, RMSE: 0.7705, MAE: 0.5962, MAPE: 21.58%
  --> 개선됨. 모델 저장됨 (RMSE: 0.7705)

Epoch 6 | Train Loss: 0.4509 | Val MSE: 0.5939, RMSE: 0.7706, MAE: 0.5932, MAPE: 20.58%
  --> 개선 없음. (1/5)

Epoch 7 | Train Loss: 0.4187 | Val MSE: 0.5962, RMSE: 0.7722, MAE: 0.5934, MAPE: 21.08%
  --> 개선 없음. (2/5)

Epoch 8 | Train Loss: 0.3870 | Val MSE: 0.6036, RMSE: 0.7769, MAE: 0.5972, MAPE: 20.87%
  --> 개선 없음. (3/5)

Epoch 9 | Train Loss: 0.3572 | Val MSE: 0.6260, RMSE: